# 10 Minutes to pandas

This is a short introduction to pandas, geared mainly for new users. You can see more complex recipes in the [Cookbook](http://pandas.pydata.org/pandas-docs/stable/cookbook.html#cookbook)

Customarily, we import as follows:

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Object Creation

See the [Data Structure Intro section](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#dsintro)

Creating a Series by passing a list of values, letting pandas create a default integer index:

In [ ]:
s = pd.Series([1,3,5,np.nan,6,8])
s

Creating a DataFrame by passing a numpy array, with a datetime index and labeled columns:

In [ ]:
dates = pd.date_range(start='20130101', periods=6)
print(dates)
df = pd.DataFrame(np.random.randn(6,4), index=dates, columns=list('ABCD'))
df

Creating a DataFrame by passing a dict of objects that can be converted to series-like.

In [ ]:
df2 = pd.DataFrame({ 'A' : 1.0, 
                     'B' : pd.Timestamp('20130102'),
                     'C' : pd.Series(2, index=list(range(4)), dtype='float32'),
                     'D' : np.array([3] * 4, dtype="int32"),
                     'E' : pd.Categorical(['test', 'train', 'tar', 'tooth']),
                     'F' : 'foo'})
df2

Having specific dtypes

In [ ]:
df2.dtypes

## Viewing Data

See the [Basics section](http://pandas.pydata.org/pandas-docs/stable/basics.html#basics)

See the top & bottom rows of the frame

In [ ]:
print(df.head())
print(df.tail(3))

Display the index, columns, and the underlying numpy data

In [ ]:
print(df.index)
print(df.columns)
print(df.values)

Describe shows a quick statistic summary of your data

In [ ]:
df.describe()

Transposing your data

In [ ]:
df.T

Sorting by an axis

axis = 0 : sort by each column
<br>
axis = 1 : sort by each row

In [ ]:
# axis = 1 means sort by columns, and desc
df.sort_index(axis=1, ascending=False)

Sorting by values

In [ ]:
df.sort_values(by='B')

## Selection

See the indexing documentation [Indexing and Selecting Data](http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing) and [MultiIndex / Advanced Indexing](http://pandas.pydata.org/pandas-docs/stable/advanced.html#advanced)

### Getting

Selecting a single column, which yields a Series, equivalent to df.A

In [ ]:
df['A']

Selecting via [], which slices the rows.

In [ ]:
print(df[0:3])
print(df['20130102':'20130104'])

### Selection by Label

See more in [Selection by Label](http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-label)

For getting a cross section using a label

In [ ]:
print(dates)
df.loc[dates[0]]

Selecting on a multi-axis by label

In [ ]:
df.loc[:,['A', 'B']]

Showing label slicing, both endpoints are included

In [ ]:
df.loc['20130101':'20130104', ['A', 'B']]

Reduction in the dimensions of the returned object

In [ ]:
df.loc['20130102', ['A', 'B']]

For getting a scalar value

In [ ]:
df.loc[dates[0], 'A']

For getting fast access to a scalar (equiv to the prior method)

In [ ]:
df.at[dates[0], 'A']

### Selection by Position

See more in [Selection by Position](http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-integer)

Select via the position of the passed integers

In [ ]:
# index starts from 0
df.iloc[3]

By integer slices, acting similar to numpy/python

In [ ]:
df.iloc[3:5, 0:2]

By lists of integer position locations, similar to the numpy/python style

In [ ]:
df.iloc[[1,2,4], [0,2]]

For slicing rows explicitly

In [ ]:
df.iloc[1:3, :]

For slicing columns explicitly

In [ ]:
df.iloc[:,1:3]

For getting a value explicitly

In [ ]:
df.iloc[1,1]

For getting fast access to a scalar (equiv to the prior method)

In [ ]:
df.iat[1,1]

### Boolean Indexing

Using a single column’s values to select data.

In [ ]:
df[df.A > 0]

Selecting values from a DataFrame where a boolean condition is met.

In [ ]:
df[df > 0]

Using the isin() method for filtering:

In [ ]:
df2 = df.copy()
df2['E'] = ['one', 'one', 'two', 'three', 'four', 'three']
print(df2)
df2[df2.E.isin(['two', 'four'])]

### Setting

Setting a new column automatically aligns the data by the indexes

In [ ]:
s1 = pd.Series([1,2,3,4,5,6], index = pd.date_range('20130102', periods=6))
print(s1)
df['F'] = s1
df

Setting values by label

In [ ]:
df.at[dates[0], 'A'] = 0

Setting values by position

In [ ]:
df.iat[0,1] = 0

Setting by assigning with a numpy array

In [ ]:
df.loc[:,'D'] = np.array([5] * len(df))

The result of the prior setting operations

In [ ]:
df

A where operation with setting.

In [ ]:
# all the positive entries will become negative
df2 = df.copy()
df2[df2 > 0] = -df2
df2

## Missing Data

pandas primarily uses the value np.nan to represent missing data. It is by default not included in computations. See the [Missing Data section](http://pandas.pydata.org/pandas-docs/stable/missing_data.html#missing-data)

Reindexing allows you to change/add/delete the index on a specified axis. This returns a copy of the data.

In [ ]:
print(df)
df1 = df.reindex(index=dates[0:4], columns=list(df.columns) + ['E'])
df1.loc[dates[0]:dates[1], 'E'] = 2.5
df1

To drop any rows that have missing data.

In [ ]:
df1.dropna(how = 'any')

Filling missing data

In [ ]:
df1.fillna(value=1.5)

To get the boolean mask where values are nan

In [ ]:
df1.isnull()

## Operations

See the [Basic section on Binary Ops](http://pandas.pydata.org/pandas-docs/stable/basics.html#basics-binop)

### Stats

Operations in general exclude missing data.

Performing a descriptive statistic

In [ ]:
# the value of df.loc['20130101', 'F'] is nan, so it is ommitted in the mean calculation of 'F' column
df.mean()

Same operation on the other axis

axis = 0 : sort by each column
<br>
axis = 1 : sort by each row

In [ ]:
df.mean(axis=1)

Operating with objects that have different dimensionality and need alignment. In addition, pandas automatically broadcasts along the specified dimension.

In [ ]:
s = pd.Series([1,3,5,np.nan,6,8], index=dates)
print(s)
s = s.shift(periods=2)
print(s)
df.sub(s, axis='index')

### Apply

Applying functions to the data

In [ ]:
# sum over each column
print(df)
df.apply(np.cumsum)

### Histogramming

See more at [Histogramming and Discretization](http://pandas.pydata.org/pandas-docs/stable/basics.html#basics-discretization)

In [ ]:
s = pd.Series(np.random.randint(0, 7, size=10))
s.value_counts(sort=False)

### String Methods

Series is equipped with a set of string processing methods in the str attribute that make it easy to operate on each element of the array, as in the code snippet below. Note that pattern-matching in str generally uses [regular expressions](https://docs.python.org/3.8/library/re.html) by default (and in some cases always uses them). See more at [Vectorized String Methods.](http://pandas.pydata.org/pandas-docs/stable/text.html#text-string-methods)

In [ ]:
s = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca', np.nan, 'CABA', 'dog', 'cat'])
s.str.lower()

## Merge

### Concat

pandas provides various facilities for easily combining together Series, DataFrame, and Panel objects with various kinds of set logic for the indexes and relational algebra functionality in the case of join / merge-type operations.

See the [Merging section](http://pandas.pydata.org/pandas-docs/stable/merging.html#merging)

Concatenating pandas objects together with concat():

In [ ]:
df = pd.DataFrame(np.random.randn(10,4))
print(df)

# break it into pieces
pieces = [df[:3], df[3:7], df[7:]]
pd.concat(pieces)

### Join

SQL style merges. See the [Database style joining](http://pandas.pydata.org/pandas-docs/stable/merging.html#merging-join)

In [ ]:
left = pd.DataFrame({'key': ['foo', 'foo'], 'lval': [1, 2]})
right = pd.DataFrame({'key': ['foo', 'foo'], 'rval': [4, 5]})
print(left)
print(right)
pd.merge(left, right, on='key')

Another example that can be given is:

In [ ]:
left = pd.DataFrame({'key': ['foo', 'bar'], 'lval': [1, 2]})
right = pd.DataFrame({'key': ['foo', 'bar'], 'rval': [4, 5]})
print(left)
print(right)
pd.merge(left, right, on='key')

### Append

Append rows to a dataframe. See the [Appending](http://pandas.pydata.org/pandas-docs/stable/merging.html#merging-concatenation)

In [ ]:
df = pd.DataFrame(np.random.randn(8, 4), columns=['A','B','C','D'])
print(df)
# add the 3rd row to the end
s = df.iloc[3]
df.append(s, ignore_index=True)

## Grouping

By “group by” we are referring to a process involving one or more of the following steps

* Splitting the data into groups based on some criteria
* Applying a function to each group independently
* Combining the results into a data structure

See the [Grouping section](http://pandas.pydata.org/pandas-docs/stable/groupby.html#groupby)

In [ ]:
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
   ....:                           'foo', 'bar', 'foo', 'foo'],
   ....:           'B' : ['one', 'one', 'two', 'three',
   ....:                           'two', 'two', 'one', 'three'],
   ....:           'C' : np.random.randn(8),
   ....:           'D' : np.random.randn(8)})
df

Grouping and then applying a function sum to the resulting groups.

In [ ]:
# only column 'C' and 'D' has numbers
df.groupby('A').sum()

Grouping by multiple columns forms a hierarchical index, which we then apply the function.

In [ ]:
df.groupby(['A', 'B']).sum()

## Reshaping

See the sections on [Hierarchical Indexing](http://pandas.pydata.org/pandas-docs/stable/advanced.html#advanced-hierarchical) and [Reshaping](http://pandas.pydata.org/pandas-docs/stable/reshaping.html#reshaping-stacking).

### Stack

In [ ]:
tuples = list(zip(*[['bar', 'bar', 'baz', 'baz',
   ....:                      'foo', 'foo', 'qux', 'qux'],
   ....:                     ['one', 'two', 'one', 'two',
   ....:                      'one', 'two', 'one', 'two']]))
print(tuples)
index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
print(index)
df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=['A', 'B'])
df2 = df[:4]
df2

The stack() method “compresses” a level in the DataFrame’s columns.

In [ ]:
stacked = df2.stack()
stacked

With a “stacked” DataFrame or Series (having a MultiIndex as the index), the inverse operation of stack() is unstack(), which by default unstacks the last level:

In [ ]:
print(stacked.unstack())
print(stacked.unstack(1))
print(stacked.unstack(0))

### Pivot Tables

See the section on [Pivot Tables](http://pandas.pydata.org/pandas-docs/stable/reshaping.html#reshaping-pivot).

In [ ]:
df = pd.DataFrame({'A' : ['one', 'one', 'two', 'three'] * 3,
   .....:          'B' : ['A', 'B', 'C'] * 4,
   .....:          'C' : ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'] * 2,
   .....:          'D' : np.random.randn(12),
   .....:          'E' : np.random.randn(12)})
df

We can produce pivot tables from this data very easily:

In [ ]:
pd.pivot_table(df, values='D', index=['A','B'], columns=['C'])

## Time Series

pandas has simple, powerful, and efficient functionality for performing resampling operations during frequency conversion (e.g., converting secondly data into 5-minutely data). This is extremely common in, but not limited to, financial applications. See the [Time Series section](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#timeseries)

In [ ]:
# Time perioeds increment by 1 second
rng = pd.date_range('1/1/2012', periods=100, freq='S')
print(rng)
ts = pd.Series(np.random.randint(0, 500, len(rng)), index=rng)
ts.resample('5Min').sum()

Time zone representation

In [ ]:
rng = pd.date_range('3/6/2012 00:00', periods=5, freq='D')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
print(ts)
ts_utc = ts.tz_localize('UTC')
ts_utc

Convert to another time zone

In [ ]:
ts_utc.tz_convert('Asia/Singapore')

Converting between time span representations

In [ ]:
rng = pd.date_range('1/1/2012', periods=5, freq='M')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
print(ts)
ps = ts.to_period()
print(ps)
ps.to_timestamp()

Converting between period and timestamp enables some convenient arithmetic functions to be used. In the following example, we convert a quarterly frequency with year ending in November to 9am of the end of the month following the quarter end:

In [ ]:
prng = pd.period_range('1990Q1', '2000Q4', freq='Q-NOV')
print(prng)
ts = pd.Series(np.random.randn(len(prng)), index=prng)
ts.index = (prng.asfreq('M', 'e') + 1).asfreq('H', 's') + 9
ts.head()

## Categoricals

pandas can include categorical data in a DataFrame. For full docs, see the [categorical introduction](https://pandas.pydata.org/pandas-docs/stable/categorical.html#categorical) and the [API documentation](https://pandas.pydata.org/pandas-docs/stable/api.html#api-categorical).

In [ ]:
df = pd.DataFrame({"id":[1,2,3,4,5,6], "raw_grade":['a', 'b', 'b', 'a', 'a', 'e']})

Convert the raw grades to a categorical data type.

In [ ]:
df["grade"] = df["raw_grade"].astype("category")
df['grade']

Rename the categories to more meaningful names (assigning to Series.cat.categories is inplace!)

In [ ]:
df['grade'].cat.categories = ["very good", "good", "very bad"]

Reorder the categories and simultaneously add the missing categories (methods under Series .cat return a new Series per default).

In [ ]:
df['grade'] = df['grade'].cat.set_categories(["very bad", "bad", "medium", "good", "very good"])
df['grade']

Sorting is per order in the categories, not lexical order.

In [ ]:
df.sort_values(by='grade')

Grouping by a categorical column shows also empty categories.

In [ ]:
df.groupby('grade').size()

## Plotting

[Plotting](https://pandas.pydata.org/pandas-docs/stable/visualization.html#visualization) docs.

In [ ]:
ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2000', periods=1000))
print(ts.head())
ts = ts.cumsum()
print(ts.head())
ts.plot()

On DataFrame, plot() is a convenience to plot all of the columns with labels:

In [ ]:
df = pd.DataFrame(np.random.randn(1000, 4), index=ts.index, columns=['A', 'B', 'C', 'D'])
df = df.cumsum()
plt.figure(); df.plot(); plt.legend(loc='best')

## Getting Data In/Out

### CSV

[Writing to a csv file](https://pandas.pydata.org/pandas-docs/stable/io.html#io-store-in-csv)

In [ ]:
df.to_csv('foo.csv')

[Reading from a csv file](https://pandas.pydata.org/pandas-docs/stable/io.html#io-read-csv-table)

In [ ]:
pd.read_csv('foo.csv')

### HDF5

Reading and writing to [HDFStores](https://pandas.pydata.org/pandas-docs/stable/io.html#io-hdf5)

Writing to a HDF5 Store

In [ ]:
df.to_hdf('foo.h5', 'df')

Reading from a HDF5 Store

In [ ]:
pd.read_hdf('foo.h5', 'df')

### Excel

Reading and writing to [MS Excel](https://pandas.pydata.org/pandas-docs/stable/io.html#io-excel)

Writing to an excel file

In [ ]:
df.to_excel('foo.xlsx', sheet_name='Sheet1')

Reading from an excel file

In [ ]:
pd.read_excel('foo.xlsx', 'Sheet1', index_col=None, na_values=['NA'])

## Gotchas

If you are trying an operation and you see an exception like:

In [ ]:
if pd.Series([False, True, False]):
    print("I was true")

See [Comparisons](https://pandas.pydata.org/pandas-docs/stable/basics.html#basics-compare) for an explanation and what to do.

See [Gotchas](https://pandas.pydata.org/pandas-docs/stable/gotchas.html#gotchas) as well.